In [1]:
import os
import struct
import numpy as np
from PIL import Image
from tqdm import tqdm
import subprocess
from osgeo import gdal; gdal.UseExceptions()
import warnings; warnings.filterwarnings("ignore")
from p_tqdm import p_umap
from util import recreate_dir, load_config
import shutil
#import traceback
# def recreate_dir(path):
#     if os.path.exists(path):
#         shutil.rmtree(path)
#     os.makedirs(path)
#     return path

# def load_config(path):
#     spec = importlib.util.spec_from_file_location("CFG", path)
#     CFG = importlib.util.module_from_spec(spec)
#     spec.loader.exec_module(CFG)
#     return CFG

In [2]:
DATA_DIR = "data/grodzisko_20230111"
CFG = load_config(f"{DATA_DIR}/config.py").CONVERSION

In [3]:
RJPG_DIR = f"{DATA_DIR}/rjpg"
TIF_RAW_DIR = f"{DATA_DIR}/tif_raw"
TMP_DIR = f"{DATA_DIR}/tmp"

# R-JPG to TIF

In [4]:
distance = round(CFG.DISTANCE)
humidity = round(CFG.HUMIDITY)
emissivity = round(CFG.EMISSIVITY,2)
reflection = round(CFG.REFLECTION)
assert os.path.exists(RJPG_DIR)
recreate_dir(TIF_RAW_DIR)
recreate_dir(TMP_DIR)

'data/grodzisko_20230111/tmp'

In [5]:

def rjpg2tif(file_name):
    #replace file extension to bin
    
    try:
        subprocess.check_call(['./dji_irp', '-a', 'measure', '--measurefmt', 'float32', 
                        '--distance', str(distance), '--humidity', str(humidity), 
                        '--emissivity', str(emissivity), '--reflection', str(reflection), 
                        '-s', f"{RJPG_DIR}/{file_name}", '-o', f"{TMP_DIR}/{file_name}"], 
                        stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    except subprocess.CalledProcessError as e:
        # invalid_files += 1
        # pbar.set_postfix_str(f"Invalid files: {invalid_files}")
        #print error stacktrace
        
        return False
    img = Image.open(f"{RJPG_DIR}/{file_name}")
    # decode temporary byte file to tif
    arr = np.zeros(img.size[0]*img.size[1])
    with open(f"{TMP_DIR}/{file_name}", "rb") as f:
        data = f.read()
        format = '{:d}f'.format(len(data)//4)
        arr=np.array(struct.unpack(format, data))
    arr = arr.reshape(img.size[1],img.size[0])
    im = Image.fromarray(arr)
    tif_file_name = file_name.replace('.JPG','.tif')
    im.save(f"{TIF_RAW_DIR}/{tif_file_name}")
    # copy exif data from original file to new tif
    #os.system(f"exiftool -tagsfromfile {RJPG_DIR}/{file_name} {TIF_RAW_DIR}/{tif_file_name} -overwrite_original_in_place")
    subprocess.check_call(['exiftool', '-tagsfromfile', f"{RJPG_DIR}/{file_name}", f"{TIF_RAW_DIR}/{tif_file_name}", '-overwrite_original_in_place'], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    return True


In [6]:
results = p_umap(rjpg2tif, os.listdir(RJPG_DIR))
print(f"Successfully converted {sum(results)} files out of {len(results)}")

  0%|          | 0/1211 [00:00<?, ?it/s]

Successfully converted 1211 files out of 1211


# Cleanup

In [ ]:
#clear temporary files
shutil.rmtree(TMP_DIR)